# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.36 which is incompatible.
jupyter-console 6.4.3 has requirement jupyter-client>=7.0.0, but you'll have jupyter-client 5.2.4 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment

#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

print (brain_name)
print (brain)

ReacherBrain
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

### 4.1 Initialization

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import Dict_Hyperparams as P
import Util as U

from collections import namedtuple, deque
from PPO_Agent import MasterAgent, Rollout, ActorAndCritic, SubNetwork

### 4.2 Training Process

The TrainingSession class trains the agent while monitoring the progress of the episodes.

In [6]:

class TrainingSession():
    
    def __init__(self, agent, num_workers):
        self.agent = agent
        self.num_workers = num_workers
        self.online_rewards = np.zeros(num_workers)
        self.mean_last_100 = 0
        self.mean_scores = []

    def train_ppo(self, agent, target_average_score, max_episodes=300):
        print("Attempting to reach 100 episode trailing average of {:.2f} in under {} episodes.".format(target_average_score, max_episodes))
        print("Rollout length: %s" % P.ROLLOUT_LENGTH)
        print("GRADIENT_CLIP %s" % P.GRADIENT_CLIP)
        print("PPO_RATIO_CLIP %s" % P.PPO_RATIO_CLIP)
        print("GAE_LAMBDA %s" % P.GAE_LAMBDA)
        #agent.save_weights()
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment   
        self.num_episodes = 0
        self.last_100_scores = deque(maxlen=100)
        self.episode_scores = [list() for i in range(num_agents)]
        self.episodes_finished = 0
        self.target_average_score = target_average_score
        while True:
            mean_last_100 = self.run_rollout(agent, env_info)
            agent.process_rollout(states)
            
            #print("\rEpisode {}\tLast 100: {:.2f}".format(self.num_episodes, mean_last_100))
            if mean_last_100 > target_average_score:
                print("Reached target! mean_last_100 %s" % mean_last_100)
                agent.save_weights() 
                break
            if self.num_episodes > max_episodes:
                print("Failed to reach target in {} episodes.".format(self.num_episodes))
                break
        return self.mean_scores 
    
    def get_actions_from_policy(self, states):
        actions = self.agent.get_prediction(states)          # Run the policy 
        actions = U.to_np(actions)                        # Extract actions
        actions = np.clip(actions, -1, 1)               # all actions between -1 and 1
        return actions
    
    def run_ppo(self, agent, max_episodes=5):
        env_info = env.reset(train_mode=False)[brain_name]
        self.last_100_scores = deque(maxlen=100)
        self.num_episodes = 0
        while True:
            mean_score_over_agents = self.run_rollout(agent, env_info)
            self.num_episodes += 1
            if self.num_episodes > max_episodes:
                print("Policy failed to reach target in %s" % max_episodes)
                break
    
    def step_environment(self, env, actions):
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment        
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished for each agent
        return (next_states, rewards, dones)

    
    '''                           
    def run_episode(self, env_info):
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        agent.start_rollout()
        while True:
            actions = self.get_actions_from_policy(states)
            next_states, rewards, dones = self.step_environment(env, actions)
    ''' 

    def run_rollout(self, agent, env_info):
        #print("Run rollout")
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        agent.start_rollout()
        for t in range(P.ROLLOUT_LENGTH):
            actions = self.get_actions_from_policy(states)            
            next_states, rewards, dones = self.step_environment(env, actions)
        
            # Save rewards
            self.online_rewards += rewards                          # Accumulate ongoing (un-normalized) rewards for each agent
            for i, done in enumerate(dones):
                if done:                                       # For a worker whose episode is done...
                    #print("Worker %s finished at timestep %s" % (i, t))
                    self.end_episode(i, self.online_rewards[i])
                    self.online_rewards[i] = 0                 # Reset accumulated reward for next episode
                    self.mean_last_100 = np.mean(self.last_100_scores)
                    if self.mean_last_100 > self.target_average_score:
                        print("Breaking")
                        break
            #print("%s step" % t)            
            agent.step(states, actions, rewards, next_states, dones) # Teach the agent 
            scores += rewards                                  # update the score (for each agent)
            states = next_states                               # roll over states to next time step

        #print("Steps in rollout: {}".format(t+1))        
        return self.mean_last_100    

    def end_episode(self, agent_index, score):
        self.episode_scores[agent_index].append(score)   # Save the reward they accumulated in the episode
        self.episodes_finished +=1
        if (self.episodes_finished % num_agents) == 0:
            self.num_episodes += 1
            total_over_agents = 0
            for i in range(num_agents):
                total_over_agents += self.episode_scores[i][-1]
            mean_score_over_agents = total_over_agents / num_agents    
            self.last_100_scores.append(mean_score_over_agents)
            self.mean_scores.append(mean_score_over_agents)
            print("Finished %s episodes (%s cycles). mean_score_over_agents %s trailing %s" % (self.num_episodes, (self.episodes_finished/num_agents), mean_score_over_agents, np.mean(self.last_100_scores)))


### 4.3 Main Program

Run the cell below to get the average scores of 20 agents.

In [ ]:
agent = MasterAgent(num_agents, state_size=state_size, action_size=action_size, seed=0, device = torch.device('cpu'))
session = TrainingSession(agent, num_agents)
scores = session.train_ppo(agent, 30.0)   # Do the training

import matplotlib.pyplot as plt
%matplotlib inline
 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Attempting to reach 100 episode trailing average of 30.00 in under 300 episodes.
Rollout length: 500
GRADIENT_CLIP 0.75
PPO_RATIO_CLIP 0.1
GAE_LAMBDA 0.95


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Finished 1 episodes (1.0 cycles). mean_score_over_agents 0.0779999982566 trailing 0.0779999982566
Finished 2 episodes (2.0 cycles). mean_score_over_agents 0.182499995921 trailing 0.130249997089
Finished 3 episodes (3.0 cycles). mean_score_over_agents 0.353499992099 trailing 0.204666662092
Finished 4 episodes (4.0 cycles). mean_score_over_agents 0.324999992736 trailing 0.234749994753
Finished 5 episodes (5.0 cycles). mean_score_over_agents 0.529999988154 trailing 0.293799993433
Finished 6 episodes (6.0 cycles). mean_score_over_agents 0.699999984354 trailing 0.36149999192
Finished 7 episodes (7.0 cycles). mean_score_over_agents 0.637499985751 trailing 0.400928562467
Finished 8 episodes (8.0 cycles). mean_score_over_agents 0.903999979794 trailing 0.463812489633
Finished 9 episodes (9.0 cycles). mean_score_over_agents 0.795999982208 trailing 0.50072221103
Finished 10 episodes (10.0 cycles). mean_score_over_agents 1.10299997535 trailing 0.560949987462
Finished 11 episodes (11.0 cycles). mea

### 4.4 Run the Policy
The code below runs the policy that has previously been trained.

In [ ]:
session.run_ppo(agent) 

In [ ]:
env.close()